In [1]:
%load_ext autoreload
%autoreload 2

import mogptk
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import qmc

#our package
from gpinit import *
import utils

plt.rcParams["figure.figsize"] = (10,3)

In [34]:
N = 1000
k = 0
input_dims = 5

mag = 0.5
l = 5.0#[0.5]*input_dims #[1.0, 0.2,  0.2,  0.2,  0.06][:input_dims]
noise = 0.5

kernel = mogptk.gpr.SquaredExponentialKernel(order=-1, input_dims=input_dims)
kernel.magnitude.assign(mag)
kernel.lengthscale.assign(l)

# halton
#x = qmc.Halton(d=input_dims).random(n=N) 
#x *= np.array([100] * input_dims)
#x *= 1+0.1*np.random.rand(x.shape[0],x.shape[1])

# random
#x = 10.0*np.random.rand(N,input_dims)

# gaussian
#x = np.random.multivariate_normal([0]*input_dims, np.diag([1]*input_dims), N)
#x = np.abs(x)

# uniform magnitude
#np.random.seed(534)
n = 100
mags = []
ls = []
noises = []
for i in range(n):
    if i%10 == 0:
        print(i, '/', n)
    x = np.random.normal(size=(N,input_dims))
    mag_ = 100.0*np.random.uniform(size=N)
    #mag_ = 100.0*np.ones((N,))
    scale_ = np.expand_dims(np.linalg.norm(x, axis=1)/mag_, axis=1)
    x /= scale_
    #print(mag, scale)
    #x = np.abs(x)

    #x = np.linspace(0, 100, N).reshape(-1,1)

    #plt.scatter(x[:,0], x[:,1])

    K = kernel(x).detach().numpy() + noise**2*np.eye(N)
    y = np.random.multivariate_normal(np.zeros((N,)), K)

    gp = gpinit(space_input=x, space_output=y,
                    aim='learning', kernel ='SE_MD', order=1)
    freqs = np.linspace(0.001,0.06,1000)
    gp.set_freqs(freqs)

    #hypers0=np.array([5, 0.966, 0.1286])
    gp.initialise(metric='L2_MD')#, hypers0=hypers0)
    mags.append(gp.theta[0]**2)
    ls.append(1.0/np.sqrt(2.0*gp.theta[1]))
    noises.append(gp.theta[2])
    
mags = np.array(mags)
ls = np.array(ls)
noises = np.array(noises)
print('magnitude:', mags.mean(), '±', mags.std())
print('lengthscale:', ls.mean(), '±', ls.std())
print('noise std.dev.:', noises.mean(), '±', noises.std())

# data = mogptk.Data(x,y)
# if input_dims == 1:
#     data.plot()

0 / 100
10 / 100
20 / 100
30 / 100
40 / 100
50 / 100
60 / 100
70 / 100
80 / 100
90 / 100
magnitude: 0.4507582870128659 ± 0.619545084558432
lengthscale: 3.9589760396274754 ± 2.501263003057776
noise std.dev.: 0.5626099523207637 ± 0.30163739994781147


In [ ]:
# x.shape

In [ ]:
# model = mogptk.Model(data, kernel)
# #print("optimal loss:", model.loss())
# print("optimal error:", model.error('mape'))

# model.gpr.kernel.magnitude.assign(np.random.rand(1))
# model.gpr.kernel.lengthscale.assign(np.random.rand(1))
# model.gpr.likelihood.scale.assign(np.random.rand(1))

# print()
# print('magnitude', model.gpr.kernel.magnitude.numpy())
# print('lengthscale', model.gpr.kernel.lengthscale.numpy())
# print('scale', model.gpr.likelihood.scale.numpy())

In [ ]:
# model.train(iters=250, method ='LBFGS', verbose=True, error='mape', lr=0.25)
# model.plot_losses();

# print()
# print('magnitude', model.gpr.kernel.magnitude.numpy())
# print('lengthscale', model.gpr.kernel.lengthscale.numpy())
# print('scale', model.gpr.likelihood.scale.numpy())

In [ ]:
# if input_dims == 1:
#     model.plot_prediction();

In [ ]:
# _, Y_true = data.get_data()
# X, Y_pred, _, _ = model.predict()

# import pandas as pd
# pd.set_option('display.max_rows', 50)

# df = pd.DataFrame()
# df['y_true'] = Y_true
# df['y_pred'] = Y_pred

# print('MAPE', np.mean(abs(df['y_true']-df['y_pred'])/df['y_true']))
# print('MSE', np.mean((df['y_true']-df['y_pred'])**2))

# df.iloc[:100]

In [ ]:
gp = gpinit(space_input=x, space_output=y,
                aim='learning', kernel ='SE_MD', order=1)
freqs = np.linspace(0.001,0.06,1000)
gp.set_freqs(freqs)

#hypers0=np.array([5, 0.966, 0.1286])
gp.initialise(metric='L2_MD')#, hypers0=hypers0)
print('magnitude:', gp.theta[0]**2)
print('lengthscale:', 1.0/np.sqrt(2.0*gp.theta[1]))
print('noise std.dev.:', gp.theta[2])

ind = np.argsort(gp.acf_times)
plt.plot(gp.acf_times[ind],gp.acf_values[ind],'o', label='Empirical variance')
plt.plot(gp.acf_times[ind], gp.Ktheta.T[ind], lw=4, label='Learned kernel (GVM)')
plt.legend()
plt.xlabel('Lag magnitude')
plt.title('Variogram for a 5-dimensional isotropic SE kernel')
plt.savefig('exp6.pdf')

In [ ]:
# model = mogptk.Model(data, kernel)
# model.gpr.kernel.magnitude.assign(gp.theta[0]**2)
# model.gpr.kernel.lengthscale.assign(1.0/np.sqrt(2.0*gp.theta[1]))
# model.gpr.likelihood.scale.assign(gp.theta[2])

# model.train(iters=250, method ='LBFGS', verbose=True, error='mape', lr=0.25)
# model.plot_losses();

# print()
# print('magnitude', model.gpr.kernel.magnitude.numpy())
# print('lengthscale', model.gpr.kernel.lengthscale.numpy())
# print('scale', model.gpr.likelihood.scale.numpy())